In [1]:
!pip install transformers

In [2]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `MyHuggingfaceAPINewOne` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token 

In [4]:
# Load an open-source LLM
model_name = "mistralai/Mistral-7B-Instruct-v0.2"  # or "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto", trust_remote_code=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [7]:

# Set up a text generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Function to do a "tree of thoughts" prompt
def tree_of_thoughts_prompt(problem_statement):
    # Step 1: Decompose the problem and generate initial thoughts
    initial_prompt = f"""You are a brilliant thinker.
You are given a problem: "{problem_statement}"

Think of 3 different initial approaches to solve this problem. Number them as 1, 2, and 3. Give a few sentences for each idea."""

    initial_thoughts = generator(initial_prompt, max_length=512, temperature=0.7, do_sample=True)[0]['generated_text']
    initial_thoughts = generator(initial_prompt, max_new_tokens=200, temperature=0.7, do_sample=True)[0]['generated_text']
    print("\n=== Initial Thoughts ===\n")
    print(initial_thoughts)

    # Step 2: Expand each thought further (next layer of tree)
    expanded_thoughts = []
    for i in range(1, 4):
        expand_prompt = f"""Building upon Thought {i} for the problem "{problem_statement}":
Expand the idea into a more detailed plan. List key steps to follow."""

        expansion = generator(expand_prompt, max_length=512, temperature=0.7, do_sample=True)[0]['generated_text']
        expanded_thoughts.append(expansion)
        print(f"\n=== Expanded Thought {i} ===\n")
        print(expansion)

    # Step 3: Evaluate and pick the best
    evaluation_prompt = f"""Here are three detailed approaches:\n1. {expanded_thoughts[0]}\n2. {expanded_thoughts[1]}\n3. {expanded_thoughts[2]}
Based on clarity, feasibility, and creativity, which approach is the best? Explain your choice."""

    evaluation = generator(evaluation_prompt, max_length=512, temperature=0.7, do_sample=True)[0]['generated_text']
    print("\n=== Final Evaluation and Selection ===\n")
    print(evaluation)



Device set to use cpu


In [8]:
# Example Problem
problem = "How to design an app that motivates users to exercise daily?"

tree_of_thoughts_prompt(problem)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== Initial Thoughts ===

You are a brilliant thinker.
You are given a problem: "How to design an app that motivates users to exercise daily?"

Think of 3 different initial approaches to solve this problem. Number them as 1, 2, and 3. Give a few sentences for each idea.

1. Social Competition: Design an app that allows users to compete with each other in terms of daily exercise goals. Users can create or join groups, set goals, and track their progress against others. The app can include features like leaderboards, badges, and rewards to further encourage competition.
2. Gamification: Design an app that turns exercise into a game. Users can earn points for completing workouts, and those points can be used to unlock achievements or rewards. The app could also include challenges or missions to complete, which can help keep users engaged and motivated.
3. Personalized Coaching: Design an app that provides users with personalized coaching and feedback based on their exercise habits and go

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== Expanded Thought 1 ===

Building upon Thought 1 for the problem "How to design an app that motivates users to exercise daily?":
Expand the idea into a more detailed plan. List key steps to follow.

1. Define the Target Audience:
   - Identify demographics (age, gender, location, etc.)
   - Determine their fitness goals and motivations
   - Understand their current exercise habits

2. Set SMART Goals:
   - Specific: What is the primary objective of the app?
   - Measurable: How will progress be tracked and reported?
   - Achievable: Is the goal realistic for the user base?
   - Relevant: Does it align with the target audience's needs and desires?
   - Time-bound: When should the goal be achieved?

3. Design the User Interface:
   - Keep it simple and intuitive
   - Use gamification elements to make exercise fun
   - Incorporate social features for accountability and community
   - Offer personalized workout plans based on user data

4. Incorporate Motivational Techniques:
   - Use 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== Expanded Thought 2 ===

Building upon Thought 2 for the problem "How to design an app that motivates users to exercise daily?":
Expand the idea into a more detailed plan. List key steps to follow.

1. Identify the target audience: Determine who the app is for and what their motivations, needs, and pain points are when it comes to exercising daily. Conduct user research, such as surveys and interviews, to gain insights.

2. Set specific, measurable, achievable, relevant, and time-bound (SMART) goals: Establish clear objectives for the app, such as increasing daily step count by 1,000 steps or encouraging users to workout for at least 30 minutes a day, five days a week.

3. Design an engaging user experience: Create an intuitive and visually appealing interface that motivates users to use the app regularly. Utilize gamification elements like badges, points, and leaderboards to incentivize users and make the experience fun.

4. Offer personalized workout plans: Tailor the exercise pl

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== Expanded Thought 3 ===

Building upon Thought 3 for the problem "How to design an app that motivates users to exercise daily?":
Expand the idea into a more detailed plan. List key steps to follow.

1. Define the target audience: Identify the demographic group that is most likely to download and use the app regularly. This could include factors such as age, gender, fitness level, and goals.

2. Research competitors: Study other fitness apps in the market and understand their strengths and weaknesses. Identify ways to differentiate the app from competitors.

3. Design the user interface: Create an intuitive and visually appealing design for the app. The interface should make it easy for users to track their progress, set goals, and access workout plans.

4. Implement gamification: Incorporate game-like elements into the app to make exercise feel more fun and rewarding. This could include earning points for workouts, competing against friends, or unlocking new features.

5. Integrate

ValueError: Input length of input_ids is 512, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.